In [1]:
import os
import tensorflow as tf
import json
from PIL import Image, ImageDraw
import numpy as np
import cv2

In [7]:
# definiowanie wymiarów obrazków treningowych
IMG_HEIGHT = 256
IMG_WIDTH = 256

In [12]:
# funkcja do wczytywania i przetwarzania obrazów i masek
def load_images_and_masks(image_file_names, mask_file_names, resize=None):
    images = []
    masks = []
    # wczytywanie obrazów i masek
    for image_file_name, mask_file_name in zip(image_file_names, mask_file_names):
        img = cv2.imread(image_file_name, cv2.IMREAD_COLOR)
        mask = cv2.imread(mask_file_name, cv2.IMREAD_GRAYSCALE)
        # sprawdzenie, czy obraz i maska mają takie same wymiary
        assert img.shape[:2] == mask.shape[:2], f"Nieprawidłowe wymiary obrazu lub maski: {image_file_name}, {mask_file_name}"
        images.append(img)
        masks.append(mask)
    # zmiana rozmiaru obrazów i masek (opcjonalne)
    if resize is not None:
        images = [cv2.resize(img, resize) for img in images]
        masks = [cv2.resize(mask, resize) for mask in masks]
    # normalizacja obrazów i masek
    images = np.array(images, dtype=np.float32) / 255.
    masks = np.array(masks, dtype=np.float32) / 255.
    # dodanie trzeciego wymiaru dla kanałów kolorów
    images = np.expand_dims(images, axis=-1)
    masks = np.expand_dims(masks, axis=-1)
    return images, masks

In [13]:
IMAGE_FOLDER_PATH = 'mixed/train/img/'
MASKS_FOLDER_PATH = 'mixed/train/mask/'
IMAGE_FOLDER_PATH_TEST = 'mixed/test/img/'
MASKS_FOLDER_PATH_TEST = 'mixed/test/mask/'

# funkcja do tworzenia listy nazw plików z obrazami i odpowiadającymi im maskami
def create_file_lists(image_folder_path, mask_folder_path):
    image_files = sorted(os.listdir(image_folder_path))
    mask_files = sorted(os.listdir(mask_folder_path))
    # upewnienie się, że listy mają tę samą liczbę plików
    assert len(image_files) == len(mask_files), "Nieprawidłowa liczba plików"
    # dodanie pełnych ścieżek do plików
    image_files = [os.path.join(image_folder_path, file_name) for file_name in image_files]
    mask_files = [os.path.join(mask_folder_path, file_name) for file_name in mask_files]
    return image_files, mask_files

# utworzenie list nazw plików dla danych treningowych i testowych
train_image_files, train_mask_files = create_file_lists(IMAGE_FOLDER_PATH, MASKS_FOLDER_PATH)
test_image_files, test_mask_files = create_file_lists(IMAGE_FOLDER_PATH_TEST, MASKS_FOLDER_PATH_TEST)

# wczytanie danych treningowych i testowych
x_train, y_train = load_images_and_masks(train_image_files, train_mask_files, resize=(256, 256))
x_test, y_test = load_images_and_masks(test_image_files, test_mask_files, resize=(256, 256))

In [14]:
def unet_small():
    # Encoder
    inputs = tf.keras.layers.Input(shape=(256, 256, 3))
    conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = tf.keras.layers.MaxPooling2D((2, 2))(conv1)
    conv2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = tf.keras.layers.MaxPooling2D((2, 2))(conv2)

    # Bottleneck
    conv3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    up1 = tf.keras.layers.UpSampling2D((2, 2))(conv3)

    # Decoder
    concat1 = tf.keras.layers.Concatenate()([conv2, up1])
    conv4 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(concat1)
    up2 = tf.keras.layers.UpSampling2D((2, 2))(conv4)
    concat2 = tf.keras.layers.Concatenate()([conv1, up2])
    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(concat2)

    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

    return model

In [15]:
model_small = unet_small()

In [16]:
model_small.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
model_small.fit(x_train, y_train, epochs=12, batch_size=4, validation_data=(x_test, y_test))

Epoch 1/12
317/317 [==============================] - 84s 264ms/step - loss: 0.5436 - accuracy: 0.7276 - val_loss: 0.4585 - val_accuracy: 0.7754
Epoch 2/12
317/317 [==============================] - 81s 254ms/step - loss: 0.4819 - accuracy: 0.7504 - val_loss: 0.4349 - val_accuracy: 0.7683
Epoch 3/12
317/317 [==============================] - 87s 274ms/step - loss: 0.4155 - accuracy: 0.7951 - val_loss: 0.3656 - val_accuracy: 0.8241
Epoch 4/12
317/317 [==============================] - 87s 274ms/step - loss: 0.4028 - accuracy: 0.8008 - val_loss: 0.3591 - val_accuracy: 0.8290
Epoch 5/12
317/317 [==============================] - 89s 279ms/step - loss: 0.3760 - accuracy: 0.8177 - val_loss: 0.3736 - val_accuracy: 0.8073
Epoch 6/12
317/317 [==============================] - 83s 261ms/step - loss: 0.3619 - accuracy: 0.8260 - val_loss: 0.4201 - val_accuracy: 0.7861
Epoch 7/12
317/317 [==============================] - 85s 267ms/step - loss: 0.3610 - accuracy: 0.8269 - val_loss: 0.3727 - val_ac

In [18]:
model_small.save_weights('unet_small_weights.h5')

In [19]:
model_small = unet_small()
model_small.load_weights('unet_small_weights.h5')

In [21]:
# Wczytaj zdjęcie i przeskaluj
img = cv2.imread('test1.png')
img = cv2.resize(img, (256, 256))

# Przekonwertuj zdjęcie na tensor
img_tensor = np.expand_dims(img, axis=0)

# Prześlij tensor przez sieć
mask_tensor = model_small.predict(img_tensor)

# Przekonwertuj maskę na obraz RGB
mask = np.squeeze(mask_tensor)
mask = np.where(mask > 0.5, 255, 0).astype('uint8')

# Wyświetl oryginalne zdjęcie i maskę
cv2.imshow('Image', img)
cv2.imshow('Mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 25ms/step
